# VotecounterTest
A script for setting up and executing performance tests on votecounters.

## Dependencies

In [1]:
import json
import time
import numpy as np
from VoteCount import VoteCount
import VoteCounter
from tqdm import tqdm

## Helper Function(s)

In [2]:
def _relevantGameInfo(game):
    link = game[:game.find('\n')]
    number = (link[link.find('&t=')+3:] if link.count('&')==1 
                      else link[link.find('&t=')+3:link.rfind('&')])
    slots, players, correct = [], [], None
    for line in game[game.find('\nPlayers\n')+9:].split('\n'):
        line = line.split(', ')
        players += line[0].split(' replaced ')
        slots.append(line[0].split(' replaced ')) 
        if (line[2].lower().count('lynched') > 0 and
            line[2].lower().count('day 1') > 0):
            correct = slots[-1]
    return slots, players, correct, number

## VotecounterTest
### Define Parameters
We should be able to play with these to select particular outputs in `VoteCounterTest` for analysis.

In [3]:
archive = '../data/archive.txt'
votecounter = VoteCounter
start_index = 0
end_index = 1

### Process Threads
Scan through each game, extract a vote sequence, and try to predict lynch with it.

In [18]:
# open game archive, separate by game
with open(archive) as f:
    games = f.read().split('\n\n\n')  
    
# process votes in each game's posts until a lynch found
# then store information about votecounter's performance
results, success, t0, total = {}, 0, time.time(), 0
end_index = end_index if end_index else len(games)  
for game in games[start_index:end_index]:
    slots, players, correct, number = _relevantGameInfo(game)
    #print(slots, players, correct, number)
    votecount = VoteCount(slots, meta={'correct': correct})
    votecounter = VoteCounter.VoteExtracter(players=players)

    # collect gameposts associated with game number
    with open('../data/posts/{}.jsonl'.format(number)) as f:
        gameposts = sorted([json.loads(l) for l in f],
                           key=lambda x: (int(x['thread']), int(x['number'])))

    tgame = time.time()
    for post in gameposts:
        # done if voters have made a choice already
        if votecount.choice:
            break

        # ignore posts not made by players
        if players.count(post['user']) == 0:
            continue

        # update votecount for each vote found by votecounter
        for voted in votecounter.fromPost(post):
            votecount.update(post['user'], voted, post['number'])
            if votecount.choice:
                success += votecount.choice == correct
                break
    results[number] = votecount
    total += 1
    print(success, total, time.time()-tgame)
    
results, success/float(end_index-start_index), time.time()-t0

1 1 0.3880000114440918


({'15787': <VoteCount.VoteCount at 0x2343297ce08>}, 1.0, 0.4790000915527344)

## Analyze Results

In [20]:
results['15787'].votelog

['Dekes voted Mariyta in post 14',
 'Dekes voted Mariyta in post 14',
 'Mr Wright voted Ant_to_the_max in post 15',
 'Mr Wright voted Ant_to_the_max in post 15',
 'AntB voted Mr Wright in post 16',
 'AntB voted Mr Wright in post 16',
 'Substrike22 voted moose200x in post 17',
 'Substrike22 voted moose200x in post 17',
 'Ant_to_the_max voted AntB in post 18',
 'Ant_to_the_max voted AntB in post 18',
 'Hiraki voted Mariyta in post 20',
 'Hiraki voted Mariyta in post 20',
 'boberz voted Hiraki in post 21',
 'boberz voted Hiraki in post 21',
 'LordChronos voted moose200x in post 32',
 'LordChronos voted moose200x in post 32',
 'Hiraki voted Empking in post 36',
 'Hiraki voted moose200x in post 36',
 'Hiraki voted Empking in post 36',
 'Hiraki voted moose200x in post 36',
 'Mariyta voted boberz in post 37',
 'Mariyta voted boberz in post 37',
 'AntB voted Empking in post 51',
 'AntB voted Empking in post 51',
 'AntB voted moose200x in post 62',
 'AntB voted moose200x in post 62',
 'Beefster